# LangChain Introduction

<img src="images/rag.jpeg" alt="RAG架構" width="45%"/>

依照上圖，RAG 的流程可以分成 5 大步驟
1. **Document Loading:** 讀取補充 LLM 知識的參考資料
2. **Splitting:** 將 Document 依照文字長短、段落、頁數等進行切分
3. **Vector Storage:** 將切分的文字轉為 Embedding 後，儲存在向量資料庫
4. **Retrival:** 找出與用戶問題相關的參考資訊
5. **Q&A:** 利用 LLM 搭配參考資料來回答用戶的問題
以下將就前 4 個步驟分別介紹

## 1. Document Loading
Langchain 裡有非常不同類型的 Document Loader，以下簡單介紹三種不同類型的 loader

其他範例可以參考：https://python.langchain.com/v0.2/docs/integrations/document_loaders/

### 1.1 PDFs

In [ ]:
from langchain.document_loaders import PyPDFLoader

fiel_dir = "PDF_DIRECTORY"


loader = PyPDFLoader(fiel_dir)
pages = loader.load()

### 1.2 YouTube Video
需要付費使用 OpenAI 的 Whisper

In [ ]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

url = "https://www.youtube.com/watch?v=jGwO_UgTS7I"
save_dir = "docs/youtube/"


loader = GenericLoader(
    YoutubeAudioLoader([url], save_dir),
    OpenAIWhisperParser())
video_docs = loader.load()
video_docs[0].page_content[0:500]

### 1.3 URLs (網路上的文章)

In [1]:
from langchain.document_loaders import WebBaseLoader

url = "https://www.bnext.com.tw/article/76864/what-is-the-meaning-of-llm"


loader = WebBaseLoader(url)
news_docs = loader.load()
news_docs[0].page_content[:100]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLLM是什麼？跟AI的關聯為何？大型語言模型要面對什麼挑戰？一文看懂|數位時代 BusinessNext\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

## 2. Splitting

理想上，如果載入資料之後不做任何處理就能給語言模型使用是最好的。

不過語言模型能接受的 tokens 數量有限(或稱 context window)，所以不免要對 Document 做進一步的處理，將 Document 切成多個更小的區塊(chunk), 這個步驟是由 LangChain 的 Text Splitters 所提供，它的大致運作過程是：

1. 將文本(text)切成多個小的且有語意的區塊(chunk)，通常是以句子為單位進行切塊，要注意過小的區塊容易喪失語意
2. 將第 1 步所產生的多個區塊進行合併，合併成所設定的區塊長度(chunk size)
3. 為了保留前後文(上下文)的情境，進一步產生含有 overlap 的區塊

畫成圖表示的話，區塊就是下列樣貌，可以看到每個區塊會有重疊(overlap)，藉此保留前後文情境：

<img src="images/chunks.png" alt="Chunk" width="25%"/>

**RecursiveCharacterTextSplitter 參數**
1. Chunk Size: 每個分割的字元數

2. Chunk Overlap: 連續的 chunk 之間重疊的字元數

3. Separator:
先以段落分割 → (若長度仍大於 chunk size) → 以換行符號分割 → (若長度仍大於 chunk size) → 以下一個分隔符號吐出

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=30)
texts_chunks = text_splitter.split_documents(news_docs)

## 3. Storage

### 3.1 Embedding

資訊有多種形式。有些資訊為非結構化資訊 (例如文字文件、多媒體和音訊)，有些則則是結構化資訊，例如應用程式日誌、資料表和圖形。

Embedding 可將所有類型的資料編碼為向量，以擷取資產含義和內容。這讓我們能夠搜尋相鄰的資料點，藉此來尋找相似的內容。

依據應用選擇合適的 Embedding 模型，可以參考 MTEB Leaderboard 挑選模型: https://huggingface.co/spaces/mteb/leaderboard

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity


# 初始化 Embedding 模型
embedding_func = HuggingFaceEmbeddings(
    model_name="infgrad/stella-base-zh-v3-1792d",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True})

# 將字句轉換為向量
a = embedding_func.embed_query('突襲式發表！蘋果推 2 款 M3 MacBook Air，強調 AI 、遊戲效能皆強化')
b = embedding_func.embed_query('蘋果最新M3版MacBook Air突襲登場！6亮點下放1技術不漲價 M2版還降3000元')

# 計算相似度
cosine_similarity([a], [b])

### 3.2 Vectore Store

向量資料庫是一種特殊類型的資料庫，專門設計用來處理、索引及搜尋非結構化資料。它不會以傳統的表格格式來組織資料，而是將資料排列成高維向量。這種獨特的結構可讓資料庫更有效率且更準確地處理複雜的多維資料。

向量資料庫的關鍵功能之一，就是使用泛型 AI 來執行分析。這包括相似性搜尋，以及異常狀況偵測，藉此找出與正常情況大不相同的資料點。

市場上存在很多不同的向量資料庫：Pinecone, Chroma, Weaviate 等

In [ ]:
from langchain.vectorstores import Chroma


# load it into Chroma
db = Chroma.from_documents(texts_chunks, embedding_func)

# query it
query = "什麼是 LLM 模型？"
docs = db.similarity_search_with_score(query)
docs[0]

## 4. Retrival

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

MODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v0_1"


# 量化參數
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True)

# llm 初始化
llm = HuggingFacePipeline.from_model_id(
    model_id=MODEL_NAME,
    task="text-generation",
    model_kwargs=dict(
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto",
        quantization_config=quantization_config),
    pipeline_kwargs=dict(
        max_new_tokens=1024,
        temperature=0.0001,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.15) )

# Prompt 模板
template = """
<s>
請你做為一個大型語言模型的專家，並根據下方所提供的資訊，來回答使用者的提問。

[INST]
{context}

{question}
[/INST]
"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])


In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

result = qa_chain({"query": query})

# LLM 提供的回答
print("回覆：")
print(result["result"])

# 參考的相關資訊
print("參考資訊：")
print(result["source_documents"])